# Mounting google drive in colab

1. Using google.colab library for mounting drive
2. This is necessary for loading all the test and train pickle files provided(alreday stored in my drive)

In [1]:
from google.colab import drive
drive.mount('/colab')

Drive already mounted at /colab; to attempt to forcibly remount, call drive.mount("/colab", force_remount=True).


Importing all the required libraries

In [2]:
import pickle
import numpy as np
import os
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Convolution2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D
from keras.models import Sequential


Using TensorFlow backend.


# Image preprocessing for trainnig and separation of validation data

1. The following code cell first loads all the respective pickle files in respective variables, and than converted to numpy nd array and reshaped the image data to shape (-1,28,28,1). Furthter we created one hot encoding of all classes using sklearn library.
2. This was need to be done so that we can feed our training data to keras model. One hot encoding was created because we need to do multiclass claassification and we need to give each class an equal weight. 


Note : No need to divide image data with 255.0 as later our BatchNormalization already do this after first convolution layer

In [11]:
# Loading training images
train_data = []
with open('/colab/My Drive/Vision_task_dataset_public/train_image.pkl','rb') as handle:
    train_data = pickle.load(handle)

# Loading  training labels   
target = []
with open('/colab/My Drive/Vision_task_dataset_public/train_label.pkl','rb') as handle:
    target = pickle.load(handle)
print(len(target))

# Converting image data to numpy nd array
train_data = np.array(train_data,dtype = np.float32)

# Reshaping images to shape(-1,28,28,1) since we need to feed this to our keras model
train_data = train_data.reshape(-1,28,28,1)

# Converting image label data to numpy nd array
target = np.array(target)

# Reshaping labels to sahpe(-1,1) so that we can feed this to sklearn OneHotEncoder which taked 2D array
target = target.reshape(-1,1)


# Importing sklearn libraray for creating One Hot Encoding and than storing it in target variable
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore',sparse = False)
target = enc.fit_transform(target)

8000


In [12]:
# Printing classes and storing them in classes variable for future use. (While converting back model predictions 
# to the previous classes)
classes = enc.categories_
classes = classes[0]
print(classes)

[0 2 3 6]


In [13]:
print(target.shape,train_data.shape)


# Importing library required for dividing training and validation data and than spliting given in 0.9 training 
# data and 0.1 validation data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data, target, test_size=0.10, random_state=42)

# Printing their shapes just to ensure correctness
print(X_train.shape,X_test.shape,y_train.shape, y_test.shape)

(8000, 4) (8000, 28, 28, 1)
(7200, 28, 28, 1) (800, 28, 28, 1) (7200, 4) (800, 4)


# Data Augmentation on training data

1. The following code cell performs data augmentation by converting one training image to total of 8 images, 
namely, same, rotated three times 90 degree, and another 4 by fliping all of them vertically
2. This was necessary since we have very small training data, we need to somehow increase this so thatour model 
learns better features, and to improve testing accuracy

In [14]:
# Importing tqdm library just to visualize progress
from tqdm import tqdm

train_data = X_train
target = y_train

# Initializing empty list to story all datas
train = []
tar = []

# Looping over every train image
for ix in tqdm(range(len(train_data))):
    temp = train_data[ix]
  
    # Storing normal image and it vertical flipped version     
    train.append(temp)
    train.append(np.flip(temp,axis=1))
  
    # Rotating image by 90 degree     
    rot = np.rot90(temp,1,(0,1))
    
    # Storing the next rotated image and its flipped version
    train.append(rot)
    train.append(np.flip(rot,axis=1))
     
    # Rotating image by 90 degree 
    rot = np.rot90(rot,1,(0,1))
    
    # Storing the next rotated image and its flipped version
    train.append(rot)
    train.append(np.flip(rot,axis=1))
  
    # Rotating image by 90 degree 
    rot = np.rot90(rot,1,(0,1))
    # Storing the next rotated image and its flipped version
    train.append(rot)
    train.append(np.flip(rot,axis=1))
  
    # Storing image labels for complete augmented data
    tar.append(target[ix])
    tar.append(target[ix])
    tar.append(target[ix])
    tar.append(target[ix])
    tar.append(target[ix])
    tar.append(target[ix])
    tar.append(target[ix])
    tar.append(target[ix])

# Printing lengths of list just for checking correctess
print(len(train))
print(len(tar))

100%|██████████| 7200/7200 [00:00<00:00, 26583.80it/s]

57600
57600


In [15]:
# Converting them to numpy nd array
X_train = np.array(train)
y_train = np.array(tar)

# Printing their shapes just to verify the correctness
print(X_train.shape,y_train.shape)

(57600, 28, 28, 1) (57600, 4)


# Designing our Keras Model

1. The following code cell lays down the structure of our keras model.
2. Our model is an example of Convolution Neural Network. I used it because it can learn image features very 
accurately and can give better accuracies

# General Approach behind Architecture

We have used specific layers to perform specific tasks
1. Convolution2D : This layer is the base of The CNN model. It creates the kernel with given size. Our model learns various features of training data
2. LeakyReLU: It is advanced activation layer for implementation of LeakyReLU
3. BatchNormalization: This is the layer which normalized data from previous layer, and this results in 10 times faster learning of our model
4. MaxPool2D: This layer helps to reduce the size by taking maximum value in the size of kernel provided and than reduces that kernel size to a single value. 
5. Flatten: This layer helps to flatten the output of previous layer to a simple 2D array
6. Dense: This layer acts like a neural network layer.

Activations Used:
1. LeakyReLU: This activation is far much better than ReLu as it provides learning when value becomes negative also.
2. Relu: This activation is also much better than other activations like tanh, sigmoid etc as it never faces problem of gradient exploding and gradient vanishing.
3. Softmax: For multiclass classification we have to use this activation.

In [16]:
model = Sequential()

model.add(Convolution2D(64, (3,3), padding='same', input_shape=(28,28,1)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3,3), padding='same'))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Convolution2D(1024, (3,3), padding='same'))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(4,activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 28, 28, 64)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 512)       295424    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 512)       0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 14, 14, 512)       2048      
__________

# Training our model and saving model weights progressively

1. The following code cell first setups the environment for training and than trains the model
2. We need to setup environment so that we can save the progressive states which are better than previous so helps to save time of re training if anything wrong goes. I keep on increasing batch size becaue it is a better approach, where instead of decreasing learning rate we used to prefer increase the batch size( A recent topic in research)

In [0]:
import keras
filepath="/colab/My Drive/model_weights/zweights-{val_acc:.4f}.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max', period=1)
callbacks_list = [checkpoint]

In [0]:
# Compiling our model
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [19]:
print(X_train.shape,X_test.shape,y_train.shape, y_test.shape)
model.fit(X_train,y_train, batch_size = 64, epochs=10,callbacks = callbacks_list, validation_data = [X_test, y_test])

(57600, 28, 28, 1) (800, 28, 28, 1) (57600, 4) (800, 4)
Instructions for updating:
Use tf.cast instead.
Train on 57600 samples, validate on 800 samples
Epoch 1/10
57600/57600 [==============================] - 72s 1ms/step - loss: 0.5112 - acc: 0.8041 - val_loss: 0.4513 - val_acc: 0.8263

Epoch 00001: val_acc improved from -inf to 0.82625, saving model to /colab/My Drive/model_weights/zweights-0.8263.h5
Epoch 2/10
57600/57600 [==============================] - 69s 1ms/step - loss: 0.3429 - acc: 0.8683 - val_loss: 0.4397 - val_acc: 0.8275

Epoch 00002: val_acc improved from 0.82625 to 0.82750, saving model to /colab/My Drive/model_weights/zweights-0.8275.h5
Epoch 3/10
57600/57600 [==============================] - 69s 1ms/step - loss: 0.2864 - acc: 0.8901 - val_loss: 0.3806 - val_acc: 0.8438

Epoch 00003: val_acc improved from 0.82750 to 0.84375, saving model to /colab/My Drive/model_weights/zweights-0.8438.h5
Epoch 4/10
57600/57600 [==============================] - 69s 1ms/step - loss

In [20]:
print(X_train.shape,X_test.shape,y_train.shape, y_test.shape)
model.fit(X_train,y_train, batch_size = 128, epochs=10,callbacks = callbacks_list, validation_data = [X_test, y_test])

(57600, 28, 28, 1) (800, 28, 28, 1) (57600, 4) (800, 4)
Train on 57600 samples, validate on 800 samples
Epoch 1/10
57600/57600 [==============================] - 53s 917us/step - loss: 0.0196 - acc: 0.9930 - val_loss: 0.6532 - val_acc: 0.8738

Epoch 00001: val_acc did not improve from 0.87625
Epoch 2/10
57600/57600 [==============================] - 52s 898us/step - loss: 0.0090 - acc: 0.9970 - val_loss: 0.6243 - val_acc: 0.8838

Epoch 00002: val_acc improved from 0.87625 to 0.88375, saving model to /colab/My Drive/model_weights/zweights-0.8838.h5
Epoch 3/10
57600/57600 [==============================] - 52s 898us/step - loss: 0.0149 - acc: 0.9947 - val_loss: 0.6894 - val_acc: 0.8662

Epoch 00003: val_acc did not improve from 0.88375
Epoch 4/10
57600/57600 [==============================] - 51s 892us/step - loss: 0.0266 - acc: 0.9902 - val_loss: 0.7239 - val_acc: 0.8612

Epoch 00004: val_acc did not improve from 0.88375
Epoch 5/10
57600/57600 [==============================] - 51s 891u

In [21]:
print(X_train.shape,X_test.shape,y_train.shape, y_test.shape)
model.fit(X_train,y_train, batch_size = 128, epochs=10,callbacks = callbacks_list, validation_data = [X_test, y_test])

(57600, 28, 28, 1) (800, 28, 28, 1) (57600, 4) (800, 4)
Train on 57600 samples, validate on 800 samples
Epoch 1/10
57600/57600 [==============================] - 51s 893us/step - loss: 0.0152 - acc: 0.9945 - val_loss: 0.8637 - val_acc: 0.8638

Epoch 00001: val_acc did not improve from 0.88375
Epoch 2/10
57600/57600 [==============================] - 51s 893us/step - loss: 0.0202 - acc: 0.9936 - val_loss: 0.8540 - val_acc: 0.8712

Epoch 00002: val_acc did not improve from 0.88375
Epoch 3/10
57600/57600 [==============================] - 51s 892us/step - loss: 0.0195 - acc: 0.9936 - val_loss: 0.7841 - val_acc: 0.8725

Epoch 00003: val_acc did not improve from 0.88375
Epoch 4/10
57600/57600 [==============================] - 51s 892us/step - loss: 0.0119 - acc: 0.9960 - val_loss: 1.0024 - val_acc: 0.8562

Epoch 00004: val_acc did not improve from 0.88375
Epoch 5/10
57600/57600 [==============================] - 51s 893us/step - loss: 0.0124 - acc: 0.9956 - val_loss: 0.8779 - val_acc: 0.86

# Loading best model weights

1. The following code cels loads best model weights from the stored weights files
2. This is necessary because it is quite possible that our model overfits the data, so we need to do early stopping by loading the best weights file with maximum val_acc

In [23]:
import os
filename="/colab/My Drive/model_weights/"
bestfile="zweights-0.0000.h5"
for file in os.listdir(filename):
    if(file.startswith("zweights-0")):
        if(bestfile<file):
            bestfile=file
print(bestfile)
model.load_weights(filename+bestfile)

zweights-0.8838.h5


# Loading test_image

In [29]:
test_data = []
with open('/colab/My Drive/Vision_task_dataset_public/test_image.pkl','rb') as handle:
    test_data = pickle.load(handle)
print(type(test_data))

<class 'list'>


In [30]:
print(len(test_data))

2000


# Making final Predictions

In [31]:
# Converting to numpy nd array
test_data = np.array(test_data)

# Reshapint test_data to shape of(-1,28,28,1) so that we can feed it to our model
test_data = test_data.reshape(-1,28,28,1)

# Making predictions
predictions = model.predict(test_data)
print(predictions.shape)

# Taking argmax of predictions as we got 4 output values against each test_data, which corresponds to each class 
# probability
maxima = np.argmax(predictions,axis=1)
print(maxima.shape)


# We have to make required csv with two columns one for index other for predicted class
final_pred = np.zeros((maxima.shape[0],2))
for ix in range(maxima.shape[0]):
    final_pred[ix][0] = int(ix)
    final_pred[ix][1] = int(classes[maxima[ix]])
print(final_pred)

(2000, 4)
(2000,)
[[0.000e+00 0.000e+00]
 [1.000e+00 0.000e+00]
 [2.000e+00 0.000e+00]
 ...
 [1.997e+03 6.000e+00]
 [1.998e+03 6.000e+00]
 [1.999e+03 6.000e+00]]


In [33]:
# Just to get intuition if predictions seems somehow valid
print(np.unique(final_pred[:,1],return_counts=True))

(array([0., 2., 3., 6.]), array([507, 498, 497, 498]))


# Storing csv file

In [0]:
import pandas as pd
pd.DataFrame(final_pred).to_csv("/colab/My Drive/Vasu_Gupta.csv",header=['image_index','class'],index=False)